In [1]:
import pathlib
from collections import Counter, defaultdict
from pprint import pprint
from io import StringIO
import os
import sys
import time
import datetime

import numpy as np
import pandas as pd
from pandas.tools.plotting import parallel_coordinates

from bs4 import BeautifulSoup
from xml.dom import minidom
from xml.etree import ElementTree as ET
#from lxml import etree as ET #Supports xpath syntax

import xmltodict 

#https://stackoverflow.com/questions/2148119/how-to-convert-an-xml-string-to-a-dictionary-in-python
#https://github.com/martinblech/xmltodict
#https://www.datacamp.com/community/tutorials/python-xml-elementtree
#https://stackabuse.com/reading-and-writing-xml-files-in-python/
#https://codereview.stackexchange.com/questions/117810/parsing-of-xml-in-python-using-minidom-and-inserting-data-into-a-database
#https://stackoverflow.com/questions/749796/pretty-printing-xml-in-python

In [2]:
pwd

'C:\\Users\\rsouza\\Documents\\Python Scripts\\exploreAT-Topothek.NLP'

Reading the XML Metadata files:

In [3]:
readme =  './TEI-XML-2018/README_English.txt'
print((open(readme, encoding="latin-1").read()))


Content of the XML files
XML are partially enriched with location data
* The location data are available as a tree in the respective entries
* Location data are available for those entries specified in helper_tables\placeName-bis Freq 10-nur Ö+STir - Tabellenblatt1.csv
* The basis are complete matches in the tag placeName

The complete place hierarchy can be found in helper_tables\listPlace.xml. This resulted in helper_tables\listPlaceBySigle.xml

There could be manual corrections to the data.


Processing Information on Places

In [4]:
placesfile =  './TEI-XML-2018/helper_tables/listPlace.xml'
#print((open(placesfile, encoding="utf-8").read()))
placestree = ET.parse(placesfile)
placesroot = placestree.getroot()
set([elem.tag for elem in placesroot.iter()])

{'{http://www.tei-c.org/ns/1.0}Bundesland',
 '{http://www.tei-c.org/ns/1.0}Gemeinde',
 '{http://www.tei-c.org/ns/1.0}Großregion',
 '{http://www.tei-c.org/ns/1.0}Kleinregion',
 '{http://www.tei-c.org/ns/1.0}Kommentar',
 '{http://www.tei-c.org/ns/1.0}Ort',
 '{http://www.tei-c.org/ns/1.0}Sigle_DB',
 '{http://www.tei-c.org/ns/1.0}idno',
 '{http://www.tei-c.org/ns/1.0}listPlace',
 '{http://www.tei-c.org/ns/1.0}note',
 '{http://www.tei-c.org/ns/1.0}place',
 '{http://www.tei-c.org/ns/1.0}placeName',
 '{http://www.tei-c.org/ns/1.0}record'}

In [5]:
placesdict = xmltodict.parse((open(placesfile, encoding="utf-8").read()))
print(placesdict.keys())
print(placesdict['listPlace'].keys())

odict_keys(['listPlace'])
odict_keys(['@xmlns', 'place'])


In [6]:
placeslist = placesdict['listPlace']['place']
state_bundesland = [p['placeName'] for p in placeslist]
print(state_bundesland)

['W.', 'STir.', 'OTir.', 'NTir.', 'Kä.', 'St.', 'Sa.', 'OÖ', 'NÖ', 'Bgl.']


In [7]:
placeslist[0]['listPlace']['place']['listPlace']['place']['listPlace']['place']['listPlace']['place'][0]['record']

OrderedDict([('Ort', 'Altmannsdorf'),
             ('Gemeinde', 'Wien'),
             ('Kleinregion', 'W.'),
             ('Großregion', 'W.'),
             ('Bundesland', 'W.'),
             ('Sigle_DB', None),
             ('Kommentar', None)])

In [8]:
placeslist[1]['listPlace']['place'][0]['listPlace']['place'][0]['listPlace']['place'][0]['listPlace']['place'][0]['record']

OrderedDict([('Ort', 'Langtaufers'),
             ('Gemeinde', 'Graun im Vinschgau'),
             ('Kleinregion', 'nördl.obVinschg.'),
             ('Großregion', 'wSTir.'),
             ('Bundesland', 'STir.'),
             ('Sigle_DB', None),
             ('Kommentar', None)])

In [9]:
placeslist[2]['listPlace']['place']['listPlace']['place'][0]['listPlace']['place'][0]['listPlace']['place']['record']

OrderedDict([('Ort', 'Abfaltersbach'),
             ('Gemeinde', 'Abfaltersbach'),
             ('Kleinregion', 'westl.OTir.Pustert.'),
             ('Großregion', 'OTir.'),
             ('Bundesland', 'OTir.'),
             ('Sigle_DB', '1B.0a01'),
             ('Kommentar', None)])

In [10]:
placeslist[3]['listPlace']['place'][0]['listPlace']['place'][0]['listPlace']['place']['listPlace']['place']['record']

OrderedDict([('Ort', 'Jungholz'),
             ('Gemeinde', 'Jungholz'),
             ('Kleinregion', 'nwAußf.'),
             ('Großregion', 'wNTir.'),
             ('Bundesland', 'NTir.'),
             ('Sigle_DB', '1C.1a01'),
             ('Kommentar', None)])

In [13]:
def show_record(something):
    while True:
        flag = False
        if type(something) == 'collections.OrderedDict' and ('record' in something.keys()):
            print(something['record']) 
            flag = True
        elif type(something) == 'collections.OrderedDict':
            something = something['listPlace']['place']
        else:
            for elem in something:
                print(something)
        if flag:
            break
        time.sleep(0.1)

In [ ]:
show_record(placeslist)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [29]:
greaterregion_Großregion_dics = [p['listPlace']['place'] for p in placesdict['listPlace']['place']][0]
greaterregion_Großregion_dics
#greaterregion_Großregion = [p['placeName'] for p in greaterregion_Großregion_dics]
#print(greaterregion_Großregion)

OrderedDict([('@type', 'Großregion'),
             ('placeName', 'W.'),
             ('idno', None),
             ('listPlace',
              OrderedDict([('place',
                            OrderedDict([('@type', 'Kleinregion'),
                                         ('placeName', 'W.'),
                                         ('idno', None),
                                         ('listPlace',
                                          OrderedDict([('place',
                                                        OrderedDict([('@type',
                                                                      'Gemeinde'),
                                                                     ('placeName',
                                                                      'Wien'),
                                                                     ('idno',
                                                                      None),
                                                  

In [25]:
print([p for p in placesdict['listPlace']['place']])

[OrderedDict([('@type', 'Bundesland'), ('placeName', 'W.'), ('idno', None), ('listPlace', OrderedDict([('place', OrderedDict([('@type', 'Großregion'), ('placeName', 'W.'), ('idno', None), ('listPlace', OrderedDict([('place', OrderedDict([('@type', 'Kleinregion'), ('placeName', 'W.'), ('idno', None), ('listPlace', OrderedDict([('place', OrderedDict([('@type', 'Gemeinde'), ('placeName', 'Wien'), ('idno', None), ('listPlace', OrderedDict([('place', [OrderedDict([('@type', 'Ort'), ('placeName', 'Altmannsdorf'), ('idno', None), ('record', OrderedDict([('Ort', 'Altmannsdorf'), ('Gemeinde', 'Wien'), ('Kleinregion', 'W.'), ('Großregion', 'W.'), ('Bundesland', 'W.'), ('Sigle_DB', None), ('Kommentar', None)]))]), OrderedDict([('@type', 'Ort'), ('placeName', 'Aspern'), ('idno', None), ('record', OrderedDict([('Ort', 'Aspern'), ('Gemeinde', 'Wien'), ('Kleinregion', 'W.'), ('Großregion', 'W.'), ('Bundesland', 'W.'), ('Sigle_DB', None), ('Kommentar', None)]))]), OrderedDict([('@type', 'Ort'), ('plac

Reading the XML Data files:

In [2]:
xmlfile =  './TEI-XML-2018/bel_d-tei-02/d157_qdb-TEI-02.xml'
#xmlfile = open(xmlfile, mode='r', encoding='utf-8').read()
namespaces = {'TEI': 'http://www.tei-c.org/ns/1.0'} # add more as needed

Parsing the XML tree

In [9]:
xmldoc = minidom.parse(xmlfile)  #xmldoc = ET.XML(xmlfile)
bs = BeautifulSoup(open(xmlfile, encoding="utf8"), 'xml')
tree = ET.parse(xmlfile)

In [10]:
root = tree.getroot()
print(root)

<Element '{http://www.tei-c.org/ns/1.0}TEI' at 0x000001D838907548>


In [11]:
my_namespaces = dict([node for _, node in ET.iterparse(StringIO(open(xmlfile, mode='r', encoding='utf-8').read()), events=['start-ns'])])
print(my_namespaces)

{'': 'http://www.tei-c.org/ns/1.0'}


In [15]:
for child in root:
    print(child.tag, child.attrib)

{http://www.tei-c.org/ns/1.0}teiHeader {}
{http://www.tei-c.org/ns/1.0}text {}


In [17]:
set([elem.tag for elem in root.iter()])

{'{http://www.tei-c.org/ns/1.0}TEI',
 '{http://www.tei-c.org/ns/1.0}att',
 '{http://www.tei-c.org/ns/1.0}bibl',
 '{http://www.tei-c.org/ns/1.0}body',
 '{http://www.tei-c.org/ns/1.0}certainty',
 '{http://www.tei-c.org/ns/1.0}cit',
 '{http://www.tei-c.org/ns/1.0}date',
 '{http://www.tei-c.org/ns/1.0}def',
 '{http://www.tei-c.org/ns/1.0}edition',
 '{http://www.tei-c.org/ns/1.0}editionStmt',
 '{http://www.tei-c.org/ns/1.0}editorialDecl',
 '{http://www.tei-c.org/ns/1.0}encodingDesc',
 '{http://www.tei-c.org/ns/1.0}entry',
 '{http://www.tei-c.org/ns/1.0}etym',
 '{http://www.tei-c.org/ns/1.0}fileDesc',
 '{http://www.tei-c.org/ns/1.0}form',
 '{http://www.tei-c.org/ns/1.0}geoDecl',
 '{http://www.tei-c.org/ns/1.0}gi',
 '{http://www.tei-c.org/ns/1.0}gram',
 '{http://www.tei-c.org/ns/1.0}gramGrp',
 '{http://www.tei-c.org/ns/1.0}idno',
 '{http://www.tei-c.org/ns/1.0}interp',
 '{http://www.tei-c.org/ns/1.0}label',
 '{http://www.tei-c.org/ns/1.0}listPlace',
 '{http://www.tei-c.org/ns/1.0}listPrefixDe

In [29]:
places = []
for elem in root.iter('{http://www.tei-c.org/ns/1.0}place'):
    # print(elem.attrib)
    # print(elem.attrib['type'])
    places.append(elem.attrib['type'])
freq_places = Counter(places)
freq_places.most_common()

[('Bundesland', 891),
 ('Großregion', 797),
 ('Kleinregion', 783),
 ('Gemeinde', 696),
 ('Ort', 696)]

In [31]:
for elem in root.iter('{http://www.tei-c.org/ns/1.0}cit'):
    print(elem.attrib)

{'type': 'kontext', 'n': '1'}
{'type': 'kontext', 'n': '1'}
{'type': 'kontext', 'n': '1'}
{'type': 'kontext', 'n': '1'}
{'type': 'kontext', 'n': '1'}
{'type': 'kontext', 'n': '1'}
{'type': 'kontext', 'n': '1'}
{'type': 'kontext', 'n': '1'}
{'type': 'kontext', 'n': '1'}
{'type': 'kontext', 'n': '1'}
{'type': 'kontext', 'n': '2'}
{'type': 'kontext', 'n': '1'}
{'type': 'kontext', 'n': '1'}
{'type': 'kontext', 'n': '1'}
{'type': 'kontext', 'n': '1'}
{'type': 'kontext', 'n': '1'}
{'type': 'kontext', 'n': '1'}
{'type': 'kontext', 'n': '1'}
{'type': 'kontext', 'n': '2'}
{'type': 'kontext', 'n': '1'}
{'type': 'kontext', 'n': '1'}
{'type': 'kontext', 'n': '1'}
{'type': 'kontext', 'n': '1'}
{'type': 'kontext', 'n': '1'}
{'type': 'kontext', 'n': '1'}
{'type': 'kontext', 'n': '2'}
{'type': 'kontext', 'n': '1'}
{'type': 'kontext', 'n': '1'}
{'type': 'kontext', 'n': '1'}
{'type': 'kontext', 'n': '1'}
{'type': 'kontext', 'n': '1'}
{'type': 'kontext', 'n': '1'}
{'type': 'kontext', 'n': '2'}
{'type': '